## 演讲文稿

请执行下面的单元格，观看讲师将用于讲解的幻灯片。

In [1]:
%%html
<iframe src="https://view.officeapps.live.com/op/embed.aspx?src=https://developer.download.nvidia.com:443/training/courses/C-MG-01-V3-ZH/task1/2-v2.pptx" width="610px" height="367px" frameborder="0"></iframe>

# 一个更真实的模型

到目前为止，您已经了解了梯度下降（即批量等于完整的数据集）和小批量随机梯度下降（批量比完整数据集小，而且通常小得多）之间的区别。您已经看到，较小的批量会增加优化过程的噪音，这有助于避免陷入局部最小值或在鞍点处减速。更小的批量也意味着运行反向传播和更新参数的次数也更多（步数更多）。

在本实验中，您将在更复杂的数据集和模型上试验批量大小。您将看到批量大小对 GPU 性能以及训练准确性的影响。

<img src="./images/Fashion MNIST.png"> 
## Fashion-MNIST 数据集

[Fashion-MNIST 数据集](https://github.com/zalandoresearch/fashion-mnist) 是对传统的 MNIST 数据集的回应，后者通常被称为机器学习的 “hello world”。 原始 MNIST 数据集包含 60,000 张手写数字 0-9 的图片。该数据集的缺点之一是它的简单性。一个模型在数据集上的良好表现并不意味着该模型在一组更复杂的图像上也能表现良好。

Fashion-MNIST 数据集是为了一个相对更复杂的图像分类的挑战而被创建的。它遵循与原始 MNIST 集相同的格式，具有 10 个类别和 60,000 张图像，每张 28x28 像素（加上 10,000 张测试图像）。 我们将在本练习中使用此数据集进行训练，并在下一个实验中介绍使用多个 GPU 进行训练。

<img src="./images/Fashion MNIST.png">

## 宽 ResNet 模型

我们将使用 Wide Residual Network 来在这个数据集上进行训练。它是一个卷积神经网络，被证明在图像分类挑战中表现非常出色。您可花一些时间来了解更多关于 [wide residual networks](https://arxiv.org/abs/1605.07146)以及它们的最初来源 [residual networks](https://arxiv.org/abs/1512.03385)的信息，或者学习更多的关于[卷积神经网络](https://developer.nvidia.com/discover/convolutional-neural-network)的知识。

![wideresnet](./images/wideresnet.png)

在 CNN 的早期，人们倾向采用非常深的模型（几十至几百层）。但随着计算能力的进步和算法的改进，特别是在残差块的想法被证明之后，人们又退回到具有更宽和更浅的网络，这是 WideResNet 模型系列的主要创新。我们将在下面使用的 WideResNet-16-10 以 1000 万规模的参数实现的精度可与具有更多参数的更深的网络相提并论。

## 训练我们的模型

我们将以超参数的默认值开始，在现有的数据集上运行训练。请花几分钟时间浏览 `fashion_mnist.py`，熟悉模型的训练过程。我们在本次培训中使用 PyTorch，但您从这些练习中获得的知识可以转用于其它深度学习框架。

请注意，我们在本练习中仅使用整个数据集的 1/6（10,000 张图像）进行训练。我们这样做是为了缩短训练时间，以便我们可以进行快速实验并查看批量大小的影响。 当我们开始引入多个 GPU 来加速训练时，我们将使用整个数据集。

一旦您对代码有了很好的理解，就可以执行下面的单元格来运行几个 epoch。 注意验证准确性、验证损失和 epoch 时间。

In [ ]:
!python fashion_mnist.py --epochs 5

我们即将对该文件进行一些编辑，所以让我们先制作一份该文件的副本。这样，如果您出任何错误，就可以有参考或备份。

In [ ]:
!cp fashion_mnist.py fashion_mnist_original.py

## 训练性能 - 图像/秒

衡量训练性能的一种方法是在给定的单位时间内处理了多少数据。GPU 针对并行处理进行了高度优化，训练过程的许多方面都利用了这种并行性。花点时间思考为什么批量大小可能会影响 GPU 的并行化能力，以及性能可能会发生什么变化。

在本练习中，您将实现一项功能，即报告神经网络训练时每秒处理的图像数量。然后，您将调整批量大小并观察性能（或吞吐量）受到的影响。

### 计算图像的吞吐量

PyTorch 让我们可以完全控制训练循环。我们将利用这种灵活性来计算和报告每个训练周期的以图像/秒为单位的吞吐量。请花点时间查看 `fashion_mnist.py` 中的 `TODO Step 1` 位置并实现图像吞吐量的计算。

实现该功能后，再次执行训练并注意图像吞吐量。

如果您遇到困难，请随时参考 [solutions/fashion_mnist_after_step_01.py](solutions/fashion_mnist_after_step_01.py)。

In [ ]:
!python fashion_mnist.py --epochs 3

您会注意到吞吐量在第一个训练周期后会增加 这可以归因于一次性成本，例如数据加载和内存分配的时间。对于下一个练习，请只关注第二个周期及以后的吞吐量。

### 按批量大小比较吞吐量

在本练习中，您将计算训练吞吐量作为批量大小的函数。请多次执行下一个单元格，每次都调整批量大小：在下面的单元格中输入数据（将每个`FIXME`替换为对应批量大小的图像/秒吞吐量），然后执行再下一个代码单元，以查看数据的图形。

In [ ]:
!python fashion_mnist.py --epochs 5 --batch-size 8

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt

data = [('8', 322),
        ('16', 485),
        ('32', 575),
        ('64', 628),
        ('128', 652),
        ('256', 657),
        ('512', 680),
        ('700', 683)]

x,y = zip(*data)
plt.bar(x,y)
plt.ylabel("Throughput (images / sec)")
plt.xlabel("Batch Size")
plt.show()

如果您不想手动查找上面的每个数据点，那么可以打开下面的代码块并复制那里提供的数据。这些值可能与您使用自己的代码生成的值略有不同，但它们是在与您当前使用的相同计算环境中生成的。

```python
data = [('8', 322),
        ('16', 485),
        ('32', 575),
        ('64', 628),
        ('128', 652),
        ('256', 657),
        ('512', 680),
        ('700', 683)]
```

请花一些时间查看数据并思考可能发生的情况。一旦您有了一些假设，请展示下面的内容（打开三个点）。

很明显，吞吐量随着批量大小的增加而增加。由于 GPU 的并行处理特性，这是合理的。更大的批量意味着，利用 GPU 中的数千个内核，我们可以并行运行更多的图像穿过模型并在执行反向传播之前计算出损失。

然而，吞吐量不会随着批量的大小线性增加，而是随着批量大小的增加而收益将递减。最终，GPU 的计算能力将饱和。当 GPU 可以同时生成数万或数十万个线程时，它们可以有效地工作，但对于小批量来说，没有足够的工作来使 GPU 能执行所有的线程。由于 GPU 通过执行大量的工作来隐藏数据延迟从而提升性能，因此小批量的性能会相对较差，而足够大的批量将有效地使用所有 GPU 内核，最终吞吐量（每秒处理的图像数量） ) 将接近天花板。

## 训练的性能：达到期望的准确率所花的时间

此时，您可能希望选择最大的批量大小进行训练，以实现最高的吞吐量。然而，虽然吞吐量是衡量训练过程的重要指标，但这并不表示模型在针对其最终目的方面（即推理）效果的好坏。

在我们的例子中，模型的好坏取决于它在给定图像中正确识别服装类别的能力。我们对这个能力的测量是根据验证集的准确率，它反映了模型对没有参与训练的独立数据集进行预测的有效性。

请思考批量大小如何影响模型的准确性。请记住，较小批量引入的噪声是训练过程中的有用工具。

在下一个练习中，您将再次调整批量大小，比较达到给定准确度之前的总训练时间。

### 提早中止

要实施提早中止，我们必须指定目标准确度和耐心值。耐心值决定了在训练停止之前应该有多少个周期超过目标准确度。有时，在网络得到有效训练之前，验证准确性可能会意外飙升。在一个以上的训练周期都保持高精度，让我们对网络受到良好的训练并且可以有效地泛化更有信心。

请在 fashion_mnist.py 中实现提早中止功能， 你可查找`TODO Step 2`。 最后，使用给定的目标精度和耐心值运行下面的训练。如果你遇到困难，你可以查看 `solutions/fashion_mnist_after_step_02.py`。

In [ ]:
!python fashion_mnist.py --target-accuracy .82 --patience 2

### 报告总的训练时间

现在您已让训练在达到一定准确度后停止，下一步是实现报告总的训练时间，以便您可以比较不同轮的运行。请在`fashion_mnist.py`中查找`TODO Step 3`。 如果你遇到困难，你可以查看 `solutions/fashion_mnist_after_step_03.py`。

完成后，请再次运行文件并测试功能。对于本次练习，您可以使用较低的目标准确度或使用较低的耐心阈值，因为我们只是要确保我们的代码正确。

In [ ]:
!python fashion_mnist.py --target-accuracy FIXME --patience FIXME

### 比较精度与批量大小

您现在有了一个系统，可以根据时间将批量大小的有效性与特定的精度（我们建议在 0.82 和 0.85 之间）进行比较。请尝试几个批量大小以查看对验证准确性的影响。注意当您的批量大小特别低或特别高时会发生什么。请考虑以相同的批量大小重复训练一次或多次，以保证评估结果的一致性。

In [ ]:
!python fashion_mnist.py --batch-size FIXME --target-accuracy FIXME --patience FIXME

在展示下一部分之前请记录并思考您的结果。

您获得的结果可能为您指明了几个大体的方向。特别是，非常小或大的批量可能不是收敛的最佳选择（非常小的批量往往噪声太大而无法充分收敛到损失函数的最小值，而非常大的批量往往使训练在早期就开始发散 )。但是，您也可能看到结果中存在很多随机性，并且很难很好地概括。没关系，这实际上是一件好事，因为并非您今天学到的所有内容都会以相同的方式应用于每个模型和数据集。本课程的目标是建立关于如何思考神经网络优化过程的直觉，而不是学习一组规则并无论青红皂白就直接套用于生产。

## 结论

在这部分课程中，我们学习了：

- 如何训练比我们之前使用的模型更复杂、更真实的神经网络模型
- 如何在 PyTorch 中实现多个自定义功能并在准确性和吞吐量方面衡量训练性能
- 批量大小如何影响更真实模型的训练准确性

实验 1 到此结束。在实验 2 中，我们将学习如何将此训练过程扩展到多个 GPU 上。

### 演讲文稿

在结束本节之前，请执行下面的单元格以查看此时讲师使用的幻灯片演示。

In [ ]:
%%html
<iframe src="https://view.officeapps.live.com/op/embed.aspx?src=https://developer.download.nvidia.com:443/training/courses/C-MG-01-V3-ZH/task1/3-v1.pptx" width="610px" height="367px" frameborder="0"></iframe>